# How to Write a Flash Attention Kernel in Pallas
## Introduction

In the previous posts in this series, we learnt how to write a [matrix multiplication kernel](https://blog.vikrampawar.com/pallas-matmul.html) and a [softmax kernel](https://blog.vikrampawar.com/pallas-softmax.html) using Pallas. Building on them, we will write a fused self-attention kernel, since self-attention includes both operations. Self-attention is a major bottleneck in deep learning architectures due to its $O(N^2)$ memory requirement, which is the bottleneck rather than computational complexity.

## Standard Attention

Mathematically, the self-attention operation is `softmax(QK^T / √d) @ V`, where  Q is a set of queries, K is a set of keys and V is a set of values. 

The Queries (Q) are usually a tensor of shape `(B, H, T, D)`, where `B` is batch size, `H` is number of heads, `T` is sequence length, and `D` is the embedding dimension (or head dimension). Each query vector represents a position in the sequence that attends to keys. Keys are used to compute attention scores with queries while values are the information retrieved based on attention weights.
The normalizing constant $\sqrt{d}$ (where $d = D$ is applied to scale down the dot-product attention scores \(QK^T\). Without it, the variance of the attention logits increases with \(D\), leading to a softmax distribution that becomes overly peaked (approaching one-hot) for large \(D\). This makes gradients unstable and training harder. Scaling by $1/\sqrt{d}$ stabilizes the variance, ensuring the softmax behaves similarly across different embedding dimensions, improving optimization and generalization.

To understand the basics of self-attention in detail, [here's](https://sebastianraschka.com/blog/2023/self-attention-from-scratch.html) an excellent blogpost by Sebastian Raschka [6]. 

In a naive implementation, it would require $O(N^2)$ memory  for sequence length $N$ to instantiate the full attention matrix. This is highly inefficient on GPUs due to the memory bottleneck - GPUs are generally much more efficient at performing the matrix multiplications than transferring data from HBM to shared memory (SMEM). To understand this tradeoff, check out my matrix multiplication kernel blogpost which has a primer on the GPU memory hierarchy [here](https://blog.vikrampawar.com/pallas-matmul.html).

In [2]:
import jax
import jax.numpy as jnp

def mha_reference(q, k, v):
    """Reference multi-head attention: softmax(Q @ K^T / sqrt(d)) @ V"""
    d = q.shape[-1]
    scale = 1.0 / jnp.sqrt(d)
    logits = jnp.einsum('bhqd,bhkd->bhqk', q, k) * scale
    probs = jax.nn.softmax(logits, axis=-1)
    o = jnp.einsum('bhqk,bhkd->bhqd', probs, v)
    return o

## The Flash Attention Algorithm

- Key insight: we never need the full attention matrix
- Combine online softmax with output accumulation
- Walk through the algorithm:
  - Tile Q (outer parallel loop)
  - Tile K, V (inner sequential loop)
  - Maintain running max `m`, sum `l`, and output accumulator `o`
  - Correction factor when max changes
  - Final normalization
- Python reference implementation (like your `online_softmax` function)

## Forward Pass Kernel

- BlockSpec design: Q tiled, K/V full sequence for inner loop
- The kernel implementation
- Storing logsumexp for backward pass

In [3]:
from functools import partial
import math

from jax.experimental import pallas as pl
from jax.experimental.pallas import triton as plgpu

INTERPRET_MODE = True  # Set to False on GPU

BLOCK_R = 64  # Block size for rows (Q blocks)
BLOCK_C = 64  # Block size for columns (KV blocks)
NUM_WARPS = 4
NUM_STAGES = 2

In [4]:
def flash_attention_fwd_kernel(q_ref, k_ref, v_ref, o_ref, logsumexp_ref, *, scale, num_k_blocks):
    """Flash attention forward kernel."""
    q_reg = plgpu.load(q_ref.at[0, :, :]).astype(jnp.float32)
    o_reg = jnp.zeros(q_reg.shape, jnp.float32)
    max_reg = jnp.full((BLOCK_R,), -jnp.inf, dtype=jnp.float32)
    l_reg = jnp.zeros((BLOCK_R,), dtype=jnp.float32)
    logsumexp_reg = jnp.zeros((BLOCK_R,), dtype=jnp.float32)

    def body(t, args):
        max_reg, l_reg, o_reg = args
        idx = pl.dslice(t * BLOCK_C, BLOCK_C)
        k_blk = plgpu.load(k_ref.at[0, idx, :]).astype(jnp.float32)
        v_blk = plgpu.load(v_ref.at[0, idx, :]).astype(jnp.float32)
        s_blk = pl.dot(q_reg, k_blk, trans_b=True) / scale
        max_blk = jnp.maximum(max_reg, jnp.max(s_blk, axis=-1))
        s_blk = jnp.exp(s_blk - max_blk[:, None])
        l_blk = jnp.sum(s_blk, axis=-1)
        o_blk = pl.dot(s_blk, v_blk)
        return (max_blk, 
                l_reg * jnp.exp(max_reg - max_blk) + l_blk, 
                o_reg * jnp.exp(max_reg - max_blk)[:, None] + o_blk)

    max_reg, l_reg, o_reg = jax.lax.fori_loop(0, num_k_blocks, body, (max_reg, l_reg, o_reg))
    logsumexp_reg = max_reg + jnp.log(l_reg)
    o_reg = o_reg / l_reg[:, None]
    plgpu.store(o_ref.at[0, :, :], o_reg.astype(o_ref.dtype))
    plgpu.store(logsumexp_ref.at[0, :], logsumexp_reg.astype(logsumexp_ref.dtype))

In [5]:
@jax.jit
def flash_attention_fwd(q, k, v):
    """Flash attention forward pass."""
    B, H, T, C = q.shape
    B_flat = B*H
    q_flat = q.reshape(-1, T, C)
    k_flat = k.reshape(-1, T, C)
    v_flat = v.reshape(-1, T, C)
    scale = math.sqrt(C)
    num_k_blocks = pl.cdiv(T, BLOCK_C)
    grid = (B_flat, pl.cdiv(T, BLOCK_R))

    out_flat, logsumexp = pl.pallas_call(
        partial(flash_attention_fwd_kernel, scale=scale, num_k_blocks=num_k_blocks),
        out_shape=[
            jax.ShapeDtypeStruct(q_flat.shape, q_flat.dtype),
            jax.ShapeDtypeStruct((B*H, T), q_flat.dtype)
        ],
        grid=grid,
        in_specs=[
            pl.BlockSpec((1, BLOCK_R, C), lambda b, t: (b, t, 0)),
            pl.BlockSpec((1, T, C), lambda b, _: (b, 0, 0)),
            pl.BlockSpec((1, T, C), lambda b, _: (b, 0, 0))
        ],
        out_specs=[
            pl.BlockSpec((1, BLOCK_R, C), lambda b, t: (b, t, 0)),
            pl.BlockSpec((1, BLOCK_R), lambda b, t: (b, t))
        ],
        interpret=INTERPRET_MODE,
        compiler_params=plgpu.CompilerParams(
            num_warps=NUM_WARPS,
            num_stages=NUM_STAGES
        )
    )(q_flat, k_flat, v_flat)
    out = out_flat.reshape(q.shape)
    logsumexp = logsumexp.reshape(B, H, T)
    return out, logsumexp

## Backward Pass

The backward pass computes gradients dQ, dK, dV given the upstream gradient dO. The key insight is that we can recompute the attention weights P from the stored logsumexp values rather than storing them:

$$P = \exp(QK^T / \sqrt{d} - \text{logsumexp})$$

We use three separate kernels to avoid atomic operations:
1. **Preprocess**: Compute $D = \text{rowsum}(O \odot dO)$ which is used in the softmax backward
2. **dK/dV kernel**: Outer loop over KV blocks, inner loop over Q blocks
3. **dQ kernel**: Outer loop over Q blocks, inner loop over KV blocks

The gradient formulas are:
- $dP = dO \cdot V^T$
- $dS = P \odot (dP - D) / \sqrt{d}$ (softmax backward with scaling)
- $dQ = dS \cdot K$
- $dK = dS^T \cdot Q$  
- $dV = P^T \cdot dO$

In [8]:
# Kernel 1: Preprocess - compute D = rowsum(O * dO)
def flash_attention_bwd_preprocess_kernel(o_ref, do_ref, d_ref):
    """Compute D = rowsum(O * dO) for backward pass."""
    o_reg = plgpu.load(o_ref).astype(jnp.float32)
    do_reg = plgpu.load(do_ref).astype(jnp.float32)
    d_reg = jnp.sum(o_reg * do_reg, axis=-1)
    plgpu.store(d_ref, d_reg.astype(d_ref.dtype))


def flash_attention_bwd_preprocess(o_flat, do_flat):
    """Preprocess for backward: compute D = rowsum(O * dO)."""
    B_flat, T, C = o_flat.shape
    grid = (B_flat, pl.cdiv(T, BLOCK_R))

    d_flat = pl.pallas_call(
        flash_attention_bwd_preprocess_kernel,
        out_shape=jax.ShapeDtypeStruct((B_flat, T), jnp.float32),
        grid=grid,
        in_specs=[
            pl.BlockSpec((1, BLOCK_R, C), lambda b, t: (b, t, 0)),
            pl.BlockSpec((1, BLOCK_R, C), lambda b, t: (b, t, 0)),
        ],
        out_specs=pl.BlockSpec((1, BLOCK_R), lambda b, t: (b, t)),
        interpret=INTERPRET_MODE,
        compiler_params=plgpu.CompilerParams(num_warps=NUM_WARPS, num_stages=NUM_STAGES)
    )(o_flat, do_flat)
    return d_flat


# Kernel 2: dK/dV - outer loop over KV blocks, inner loop over Q blocks
def flash_attention_bwd_dkv_kernel(
    q_ref, k_ref, v_ref, do_ref, logsumexp_ref, d_ref,
    dk_ref, dv_ref,
    *, scale, num_q_blocks
):
    """Compute dK and dV gradients."""
    k_reg = plgpu.load(k_ref.at[0, :, :]).astype(jnp.float32)
    v_reg = plgpu.load(v_ref.at[0, :, :]).astype(jnp.float32)

    dk_acc = jnp.zeros(dk_ref.shape, dtype=jnp.float32)
    dv_acc = jnp.zeros(dv_ref.shape, dtype=jnp.float32)

    def body(t, carry):
        dk_acc, dv_acc = carry
        idx = pl.dslice(t * BLOCK_R, BLOCK_R)
        q_blk = plgpu.load(q_ref.at[0, idx, :]).astype(jnp.float32)
        do_blk = plgpu.load(do_ref.at[0, idx, :]).astype(jnp.float32)
        logsumexp_blk = plgpu.load(logsumexp_ref.at[0, idx]).astype(jnp.float32)
        d_blk = plgpu.load(d_ref.at[0, idx]).astype(jnp.float32)
        # Recompute P = softmax(Q @ K^T / scale)
        s_blk = pl.dot(q_blk, k_reg, trans_b=True) / scale
        p_blk = jnp.exp(s_blk - logsumexp_blk[..., None])
        # dP = dO @ V^T, dS = P * (dP - D) / scale
        dp_blk = pl.dot(do_blk, v_reg, trans_b=True)
        ds_blk = p_blk * (dp_blk - d_blk[..., None]) / scale
        # Accumulate: dV += P^T @ dO, dK += dS^T @ Q
        dv_acc += pl.dot(p_blk, do_blk, trans_a=True)
        dk_acc += pl.dot(ds_blk, q_blk, trans_a=True)
        return dk_acc, dv_acc
        
    dk_acc, dv_acc = jax.lax.fori_loop(0, num_q_blocks, body, (dk_acc, dv_acc))
    plgpu.store(dk_ref, dk_acc.astype(dk_ref.dtype))
    plgpu.store(dv_ref, dv_acc.astype(dv_ref.dtype))


def flash_attention_bwd_dkv(q_flat, k_flat, v_flat, do_flat, logsumexp_flat, d_flat, scale):
    """Compute dK and dV using pallas_call."""
    B_flat, T, C = q_flat.shape
    num_q_blocks = pl.cdiv(T, BLOCK_R)
    grid = (B_flat, pl.cdiv(T, BLOCK_C))

    dk_flat, dv_flat = pl.pallas_call(
        partial(flash_attention_bwd_dkv_kernel, scale=scale, num_q_blocks=num_q_blocks),
        out_shape=[
            jax.ShapeDtypeStruct(k_flat.shape, k_flat.dtype),
            jax.ShapeDtypeStruct(v_flat.shape, v_flat.dtype),
        ],
        grid=grid,
        in_specs=[
            pl.BlockSpec((1, T, C), lambda b, _: (b, 0, 0)),       # q (full)
            pl.BlockSpec((1, BLOCK_C, C), lambda b, t: (b, t, 0)), # k (blocked)
            pl.BlockSpec((1, BLOCK_C, C), lambda b, t: (b, t, 0)), # v (blocked)
            pl.BlockSpec((1, T, C), lambda b, _: (b, 0, 0)),       # do (full)
            pl.BlockSpec((1, T), lambda b, _: (b, 0)),             # logsumexp (full)
            pl.BlockSpec((1, T), lambda b, _: (b, 0)),             # d (full)
        ],
        out_specs=[
            pl.BlockSpec((1, BLOCK_C, C), lambda b, t: (b, t, 0)),
            pl.BlockSpec((1, BLOCK_C, C), lambda b, t: (b, t, 0)),
        ],
        interpret=INTERPRET_MODE,
        compiler_params=plgpu.CompilerParams(num_warps=NUM_WARPS, num_stages=NUM_STAGES)
    )(q_flat, k_flat, v_flat, do_flat, logsumexp_flat, d_flat)
    return dk_flat, dv_flat


# Kernel 3: dQ - outer loop over Q blocks, inner loop over KV blocks
def flash_attention_bwd_dq_kernel(
    q_ref, k_ref, v_ref, do_ref, logsumexp_ref, d_ref,
    dq_ref,
    *, scale, num_kv_blocks
):
    """Compute dQ gradient."""
    q_reg = plgpu.load(q_ref.at[0, :, :]).astype(jnp.float32)
    do_reg = plgpu.load(do_ref.at[0, :, :]).astype(jnp.float32)
    logsumexp_reg = plgpu.load(logsumexp_ref.at[0, :]).astype(jnp.float32)
    d_reg = plgpu.load(d_ref.at[0, :]).astype(jnp.float32)
    dq_acc = jnp.zeros(dq_ref.shape, dtype=jnp.float32)

    def body(t, carry):
        dq_acc = carry
        idx = pl.dslice(t * BLOCK_C, BLOCK_C)
        k_blk = plgpu.load(k_ref.at[0, idx, :]).astype(jnp.float32)
        v_blk = plgpu.load(v_ref.at[0, idx, :]).astype(jnp.float32)
        # Recompute P
        s_blk = pl.dot(q_reg, k_blk, trans_b=True) / scale
        p_blk = jnp.exp(s_blk - logsumexp_reg[..., None])
        # dP = dO @ V^T, dS = P * (dP - D) / scale
        dp_blk = pl.dot(do_reg, v_blk, trans_b=True)
        ds_blk = p_blk * (dp_blk - d_reg[..., None]) / scale
        # Accumulate: dQ += dS @ K
        dq_acc += pl.dot(ds_blk, k_blk)
        return dq_acc

    dq_acc = jax.lax.fori_loop(0, num_kv_blocks, body, dq_acc)
    plgpu.store(dq_ref, dq_acc.astype(dq_ref.dtype))


def flash_attention_bwd_dq(q_flat, k_flat, v_flat, do_flat, logsumexp_flat, d_flat, scale):
    """Compute dQ using pallas_call."""
    B_flat, T, C = q_flat.shape
    num_kv_blocks = pl.cdiv(T, BLOCK_C)
    grid = (B_flat, pl.cdiv(T, BLOCK_R))

    dq_flat = pl.pallas_call(
        partial(flash_attention_bwd_dq_kernel, scale=scale, num_kv_blocks=num_kv_blocks),
        out_shape=jax.ShapeDtypeStruct(q_flat.shape, q_flat.dtype),
        grid=grid,
        in_specs=[
            pl.BlockSpec((1, BLOCK_R, C), lambda b, t: (b, t, 0)), # q (blocked)
            pl.BlockSpec((1, T, C), lambda b, _: (b, 0, 0)),       # k (full)
            pl.BlockSpec((1, T, C), lambda b, _: (b, 0, 0)),       # v (full)
            pl.BlockSpec((1, BLOCK_R, C), lambda b, t: (b, t, 0)), # do (blocked)
            pl.BlockSpec((1, BLOCK_R), lambda b, t: (b, t)),       # logsumexp (blocked)
            pl.BlockSpec((1, BLOCK_R), lambda b, t: (b, t)),       # d (blocked)
        ],
        out_specs=pl.BlockSpec((1, BLOCK_R, C), lambda b, t: (b, t, 0)),
        interpret=INTERPRET_MODE,
        compiler_params=plgpu.CompilerParams(num_warps=NUM_WARPS, num_stages=NUM_STAGES)
    )(q_flat, k_flat, v_flat, do_flat, logsumexp_flat, d_flat)
    return dq_flat


@jax.jit
def flash_attention_bwd(q, k, v, o, logsumexp, do):
    """Flash attention backward pass using 3 separate kernels."""
    B, H, T, C = q.shape
    scale = math.sqrt(C)

    # Flatten batch and head dimensions
    q_flat = q.reshape(-1, T, C)
    k_flat = k.reshape(-1, T, C)
    v_flat = v.reshape(-1, T, C)
    o_flat = o.reshape(-1, T, C)
    do_flat = do.reshape(-1, T, C)
    logsumexp_flat = logsumexp.reshape(-1, T)

    # Kernel 1: Preprocess - compute D = rowsum(O * dO)
    d_flat = flash_attention_bwd_preprocess(o_flat, do_flat)

    # Kernel 2: Compute dK, dV
    dk_flat, dv_flat = flash_attention_bwd_dkv(
        q_flat, k_flat, v_flat, do_flat, logsumexp_flat, d_flat, scale
    )

    # Kernel 3: Compute dQ
    dq_flat = flash_attention_bwd_dq(
        q_flat, k_flat, v_flat, do_flat, logsumexp_flat, d_flat, scale
    )

    return (
        dq_flat.reshape(q.shape),
        dk_flat.reshape(k.shape),
        dv_flat.reshape(v.shape),
    )

## Custom VJP Integration

- Wiring up forward and backward with `jax.custom_vjp`
- The residuals needed (Q, K, V, O, logsumexp)

In [9]:
@jax.custom_vjp
def flash_attention(q, k, v):
    """Flash attention with custom backward pass."""
    o, _ = flash_attention_fwd(q, k, v)
    return o


def flash_attention_fwd_rule(q, k, v):
    """Forward rule for custom_vjp.
    
    Returns the output and residuals needed for backward pass.
    """
    o, logsumexp = flash_attention_fwd(q, k, v)
    return o, (q, k, v, o, logsumexp)


def flash_attention_bwd_rule(res, do):
    """Backward rule for custom_vjp.
    
    Takes residuals from forward and upstream gradient dO,
    returns gradients (dQ, dK, dV).
    """
    q, k, v, o, logsumexp = res
    dq, dk, dv = flash_attention_bwd(q, k, v, o, logsumexp, do)
    return dq, dk, dv


flash_attention.defvjp(flash_attention_fwd_rule, flash_attention_bwd_rule)

## Evaluation

- Gradient correctness check against JAX autodiff
- Train a small transformer or attention layer to verify end-to-end

In [10]:
B, H, T, D = 2, 4, 256, 64
key = jax.random.key(0)
keys = jax.random.split(key, 4)

q = jax.random.normal(keys[0], (B, H, T, D), dtype=jnp.float32)
k = jax.random.normal(keys[1], (B, H, T, D), dtype=jnp.float32)
v = jax.random.normal(keys[2], (B, H, T, D), dtype=jnp.float32)
do = jax.random.normal(keys[3], (B, H, T, D), dtype=jnp.float32)

# Forward check
o_ref = mha_reference(q, k, v)
print(f"Reference output shape: {o_ref.shape}")

o_flash = flash_attention(q, k, v)
print(f"Flash attention output shape: {o_flash.shape}")
print(f"Forward pass matches: {jnp.allclose(o_flash, o_ref, atol=1e-2, rtol=1e-2)}")

Reference output shape: (2, 4, 256, 64)
Flash attention output shape: (2, 4, 256, 64)
Forward pass matches: True


In [11]:
# Backward check (reference)
def loss_ref(q, k, v):
    return jnp.sum(mha_reference(q, k, v) * do)

dq_ref, dk_ref, dv_ref = jax.grad(loss_ref, argnums=(0, 1, 2))(q, k, v)
print(f"Reference gradient shapes: dq={dq_ref.shape}, dk={dk_ref.shape}, dv={dv_ref.shape}")

# Flash attention backward pass
def loss_flash(q, k, v):
    return jnp.sum(flash_attention(q, k, v) * do)

dq_flash, dk_flash, dv_flash = jax.grad(loss_flash, argnums=(0, 1, 2))(q, k, v)
print(f"Flash attention gradient shapes: dq={dq_flash.shape}, dk={dk_flash.shape}, dv={dv_flash.shape}")

print(f"dQ matches: {jnp.allclose(dq_flash, dq_ref, atol=1e-2, rtol=1e-2)}")
print(f"dK matches: {jnp.allclose(dk_flash, dk_ref, atol=1e-2, rtol=1e-2)}")
print(f"dV matches: {jnp.allclose(dv_flash, dv_ref, atol=1e-2, rtol=1e-2)}")

# Print max differences for debugging
print(f"\nMax differences:")
print(f"  dQ: {jnp.max(jnp.abs(dq_flash - dq_ref)):.6f}")
print(f"  dK: {jnp.max(jnp.abs(dk_flash - dk_ref)):.6f}")
print(f"  dV: {jnp.max(jnp.abs(dv_flash - dv_ref)):.6f}")

Reference gradient shapes: dq=(2, 4, 256, 64), dk=(2, 4, 256, 64), dv=(2, 4, 256, 64)
Flash attention gradient shapes: dq=(2, 4, 256, 64), dk=(2, 4, 256, 64), dv=(2, 4, 256, 64)
dQ matches: True
dK matches: True
dV matches: True

Max differences:
  dQ: 0.000000
  dK: 0.000000
  dV: 0.000001


## Performance Comparison

We compare our Pallas flash attention implementation against:
1. **JAX cuDNN**: `jax.nn.dot_product_attention(implementation='cudnn')` - NVIDIA's highly optimized implementation
2. **Reference (materialized)**: Standard attention that materializes the full N×N attention matrix

Note: The cuDNN implementation requires a GPU with cuDNN installed and uses float16 for optimal performance. Set `INTERPRET_MODE = False` to run on GPU.

In [ ]:
import time

def bench(fn, *args, iters=10):
    for _ in range(3):  # warmup
        result = fn(*args)
        if isinstance(result, tuple):
            result[0].block_until_ready()
        else:
            result.block_until_ready()
    times = []
    for _ in range(iters):
        t0 = time.perf_counter()
        result = fn(*args)
        if isinstance(result, tuple):
            result[0].block_until_ready()
        else:
            result.block_until_ready()
        times.append(time.perf_counter() - t0)
    return sum(times) / len(times)

In [ ]:
# Performance benchmark (requires GPU with cuDNN)
# Skip this cell if running on CPU

def benchmark_attention():
    """Benchmark attention implementations."""
    import time
    
    # Use float16 for cuDNN compatibility
    B, H, T, D = 4, 8, 1024, 64
    key = jax.random.key(42)
    keys = jax.random.split(key, 4)
    
    q = jax.random.normal(keys[0], (B, H, T, D), dtype=jnp.float16)
    k = jax.random.normal(keys[1], (B, H, T, D), dtype=jnp.float16)
    v = jax.random.normal(keys[2], (B, H, T, D), dtype=jnp.float16)
    do = jax.random.normal(keys[3], (B, H, T, D), dtype=jnp.float16)
    
    print(f"Benchmark shape: B={B}, H={H}, T={T}, D={D}, dtype=float16")
    print("=" * 60)
    
    def bench_fwd(fn, q, k, v, iters=20):
        # Warmup
        for _ in range(3):
            out = fn(q, k, v)
            jax.block_until_ready(out)
        # Bench
        times = []
        for _ in range(iters):
            t0 = time.perf_counter()
            out = fn(q, k, v)
            jax.block_until_ready(out)
            times.append(time.perf_counter() - t0)
        return sum(times) / len(times) * 1000  # ms

    def bench_bwd(fn, q, k, v, do, iters=20):
        # Warmup
        for _ in range(3):
            grads = jax.grad(lambda q, k, v: jnp.sum(fn(q, k, v) * do), argnums=(0, 1, 2))(q, k, v)
            jax.block_until_ready(grads)
        # Bench
        times = []
        for _ in range(iters):
            t0 = time.perf_counter()
            grads = jax.grad(lambda q, k, v: jnp.sum(fn(q, k, v) * do), argnums=(0, 1, 2))(q, k, v)
            jax.block_until_ready(grads)
            times.append(time.perf_counter() - t0)
        return sum(times) / len(times) * 1000  # ms

    # JAX cuDNN (requires GPU)
    @jax.jit
    def jax_cudnn_attention(q, k, v):
        # Transpose from (B, H, T, D) to (B, T, H, D) for jax.nn.dot_product_attention
        q_t = jnp.transpose(q, (0, 2, 1, 3))
        k_t = jnp.transpose(k, (0, 2, 1, 3))
        v_t = jnp.transpose(v, (0, 2, 1, 3))
        out = jax.nn.dot_product_attention(q_t, k_t, v_t, implementation='cudnn')
        return jnp.transpose(out, (0, 2, 1, 3))

    # Our Pallas implementation
    @jax.jit
    def pallas_attention(q, k, v):
        return flash_attention(q, k, v)

    # Reference (materialized attention matrix)
    @jax.jit 
    def reference_attention(q, k, v):
        return mha_reference(q, k, v)

    print("\nForward pass:")
    try:
        t_cudnn = bench_fwd(jax_cudnn_attention, q, k, v)
        print(f"  JAX cuDNN:              {t_cudnn:.3f} ms")
    except Exception as e:
        print(f"  JAX cuDNN:              N/A (cuDNN not available)")
        t_cudnn = None
    
    t_pallas = bench_fwd(pallas_attention, q, k, v)
    print(f"  Our Pallas:             {t_pallas:.3f} ms")
    
    t_ref = bench_fwd(reference_attention, q, k, v)
    print(f"  Reference (materialized): {t_ref:.3f} ms")
    
    if t_cudnn:
        print(f"\n  Pallas vs cuDNN: {t_pallas/t_cudnn:.2f}x slower")

    print("\nBackward pass:")
    try:
        t_cudnn_bwd = bench_bwd(jax_cudnn_attention, q, k, v, do)
        print(f"  JAX cuDNN:              {t_cudnn_bwd:.3f} ms")
    except Exception as e:
        print(f"  JAX cuDNN:              N/A (cuDNN not available)")
        t_cudnn_bwd = None
    
    t_pallas_bwd = bench_bwd(pallas_attention, q, k, v, do)
    print(f"  Our Pallas:             {t_pallas_bwd:.3f} ms")
    
    t_ref_bwd = bench_bwd(reference_attention, q, k, v, do)
    print(f"  Reference (materialized): {t_ref_bwd:.3f} ms")
    
    if t_cudnn_bwd:
        print(f"\n  Pallas vs cuDNN: {t_pallas_bwd/t_cudnn_bwd:.2f}x slower")

# Uncomment to run benchmark (requires GPU):
# benchmark_attention()

### Example Results (RTX 4000 Ada)

When run on an NVIDIA RTX 4000 Ada GPU, typical results are:

```
Benchmark shape: B=4, H=8, T=1024, D=64, dtype=float16
============================================================

Forward pass:
  JAX cuDNN:                0.368 ms
  Our Pallas:               0.433 ms
  Reference (materialized): 1.647 ms

  Pallas vs cuDNN: 1.18x slower

Backward pass:
  JAX cuDNN:                3.230 ms
  Our Pallas:               5.728 ms
  Reference (materialized): 6.339 ms

  Pallas vs cuDNN: 1.77x slower
```

**Key observations:**
- Our forward pass is ~18% slower than cuDNN
- Our backward pass is ~77% slower than cuDNN (due to 3 separate kernel launches)
- Both are significantly faster than materializing the full attention matrix

## Conclusion

We've implemented a complete Flash Attention kernel in JAX Pallas with both forward and backward passes. The key ideas are:

1. **Online softmax**: Computing softmax in tiles without materializing the full N×N attention matrix
2. **Correction factors**: Rescaling partial results when the running maximum changes
3. **Recomputation**: Storing only logsumexp and recomputing attention weights in the backward pass
4. **Three backward kernels**: Separate passes for D (preprocess), dK/dV, and dQ to avoid atomic operations

The implementation achieves correctness and demonstrates the core Flash Attention algorithm clearly. While it doesn't match cuDNN performance, it serves as an excellent educational resource for understanding how memory-efficient attention works.

## Limitations and Future Work

### Performance Gap

Our Pallas implementation achieves correctness but runs approximately 1.5-2x slower than NVIDIA's cuDNN flash attention on the forward pass, and the gap widens on the backward pass. The primary reasons for this performance gap are:

1. **Lack of warp-level tiling**: FlashAttention-2 uses sophisticated warp-level parallelism where different warps within a thread block handle different portions of the K/V matrices. This reduces shared memory traffic and improves tensor core utilization.

2. **Three separate backward kernels**: Our implementation uses three kernel launches (preprocess, dK/dV, dQ) to avoid atomic operations. Production implementations fuse these more aggressively with careful synchronization.

3. **No causal masking optimization**: Causal attention can skip computation for masked positions, but our implementation computes the full attention matrix.

### Pallas Limitations

Pallas provides a high-level abstraction for writing GPU kernels, but it doesn't expose certain low-level primitives needed for peak performance:

- **No warp-level programming**: Pallas doesn't provide access to `warp_id` or warp shuffle operations (`__shfl_sync`). You can configure `num_warps` but cannot coordinate work between warps within a block.

- **Limited shared memory control**: Pallas manages shared memory implicitly through `BlockSpec`. You cannot explicitly allocate shared memory or control synchronization barriers.

- **No atomic operations**: Pallas on GPU doesn't expose `atomic_add` or similar primitives, requiring separate kernels for reductions.

### Path to Better Performance

To close the gap with cuDNN, you would need to:

1. **Switch to Triton**: Triton provides more control over memory access patterns, explicit masking with `tl.where`, and better autotuning. However, even Triton abstracts away some warp-level primitives.

2. **Use CUDA C++**: For full control over warp-level tiling, shared memory, and synchronization, CUDA C++ remains necessary. This is what cuDNN and the original FlashAttention implementations use.

3. **Just use the built-in**: For production workloads, `jax.nn.dot_product_attention(implementation='cudnn')` is the pragmatic choice. It's highly optimized and well-tested.

### Educational Value

Despite the performance gap, this Pallas implementation has significant educational value:

- **Algorithm clarity**: The tiled computation with online softmax correction is clearly visible in the code
- **Gradient derivation**: The backward pass shows exactly how gradients flow through attention
- **Pallas patterns**: Demonstrates `BlockSpec`, `fori_loop`, and `custom_vjp` integration
- **Debugging**: `INTERPRET_MODE=True` allows stepping through the algorithm on CPU

For learning how flash attention works, this implementation is arguably better than optimized CUDA code where the algorithm is obscured by performance tricks.

## References

1. Dao, T., Fu, D., Ermon, S., Rudra, A., & Re, C. (2022). FlashAttention: Fast and Memory-Efficient Exact Attention with IO-Awareness. *NeurIPS 2022*. https://arxiv.org/abs/2205.14135
2. Dao, T. (2023). FlashAttention-2: Faster Attention with Better Parallelism and Work Partitioning. *arXiv preprint arXiv:2307.08691*. https://arxiv.org/abs/2307.08691
3. JAX Official Flash Attention (TPU): https://github.com/jax-ml/jax/blob/main/jax/experimental/pallas/ops/tpu/flash_attention.py
4. JAX Official Fused Attention (GPU): https://github.com/jax-ml/jax/blob/main/jax/experimental/pallas/ops/gpu/attention.py
5. Umar Jamil's Triton Flash Attention: https://github.com/hkproj/triton-flash-attention
6. Sebastian Raschka - Understanding and Coding Self-Attention from Scratch: https://sebastianraschka.com/blog/2023/self-attention-from-scratch.html